<a href="https://colab.research.google.com/github/leokao960811/ProgrammingLang/blob/main/W7_TextAnalyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

###自行練習 (用nltk進行英文斷詞)

In [49]:
gsheets2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1b6WaMdMXtSRzg5PqQ9OXaw-32gAP7W0-yVhrxtsVL1A/edit?usp=sharing').get_worksheet(2)

# 讀取所有數據
rows = gsheets2.get_all_records()
import pandas as pd
df2 = pd.DataFrame(rows)

In [50]:
df2

,Text
0,i didnt feel humiliated
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...
4,i am feeling grouchy
...,...
95,i feel like throwing away the shitty piece of ...
96,im starting to feel wryly amused at the banal ...
97,i find every body beautiful and only want peop...
98,i hear are owners who feel victimized by their...


In [51]:
# 使用 nltk 斷詞
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [52]:
from nltk.tokenize import word_tokenize
def tokenize_sentence(sentence):
    tokens = word_tokenize(sentence)
    return tokens

In [53]:
df2['Tokenized'] = df2['Text'].apply(tokenize_sentence)

In [54]:
df2['Tokenized']

,Tokenized
0,"[i, didnt, feel, humiliated]"
1,"[i, can, go, from, feeling, so, hopeless, to, ..."
2,"[im, grabbing, a, minute, to, post, i, feel, g..."
3,"[i, am, ever, feeling, nostalgic, about, the, ..."
4,"[i, am, feeling, grouchy]"
...,...
95,"[i, feel, like, throwing, away, the, shitty, p..."
96,"[im, starting, to, feel, wryly, amused, at, th..."
97,"[i, find, every, body, beautiful, and, only, w..."
98,"[i, hear, are, owners, who, feel, victimized, ..."


In [55]:
all_words2 = [word for tokens in df2['Tokenized'] for word in tokens if len(word) > 1]

In [56]:
word_counts2 = Counter(all_words2)
word_counts2

Counter({'didnt': 2,
         'feel': 63,
         'humiliated': 1,
         'can': 9,
         'go': 5,
         'from': 5,
         'feeling': 35,
         'so': 16,
         'hopeless': 1,
         'to': 60,
         'damned': 1,
         'hopeful': 1,
         'just': 5,
         'being': 2,
         'around': 5,
         'someone': 6,
         'who': 5,
         'cares': 1,
         'and': 60,
         'is': 17,
         'awake': 2,
         'im': 7,
         'grabbing': 1,
         'minute': 1,
         'post': 1,
         'greedy': 2,
         'wrong': 2,
         'am': 14,
         'ever': 3,
         'nostalgic': 1,
         'about': 6,
         'the': 58,
         'fireplace': 1,
         'will': 6,
         'know': 7,
         'that': 35,
         'it': 11,
         'still': 3,
         'on': 11,
         'property': 2,
         'grouchy': 1,
         'ive': 5,
         'been': 8,
         'little': 6,
         'burdened': 1,
         'lately': 2,
         'wasnt': 1,
      

In [57]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words2 = set(['and', 'to', 'is', 'am', 'are', 'he', 'she', 'it', 'the'])
filtered_word_counts2 = {word: count for word, count in word_counts2.items() if word not in stop_words2}

In [58]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df2 = pd.DataFrame(filtered_word_counts2.items(), columns=['Word', 'Frequency'])
word_freq_df2 = word_freq_df2.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df2.head(20), x='Word', y='Frequency',
             title='字詞出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [45]:
word_freq_df2

,Word,Frequency
0,feel,63
1,feeling,35
2,that,35
3,of,31
4,in,27
...,...,...
723,banal,1
724,wryly,1
725,hopeful,1
726,damned,1


In [46]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words2 = word_freq_df2.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words2['Word'], top_words2['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [59]:
from google.colab import userdata
apikey = userdata.get('test')

import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"這是一個人在100天內的日記摘要，請摘要出這些語句的重點，並補上重點中的情緒表現，把最多次呈現的情緒及其相關的內容整併在一起，說明這個人的情緒變化，並以繁體中文回答：{df2.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這100天內的日記摘要顯示這位作者的情緒波動劇烈，沒有持續一致的情緒狀態。但若要說出現次數最多且相關的情緒，那就是**負面情緒，尤其是「沮喪」、「無助」和「焦慮」**。  這些情緒常常交織在一起出現，並與生活中的不同事件連結。

**情緒變化及相關內容：**

* **沮喪/無助 (Hopelessness/Helplessness):**  這類情緒在多篇日記中出現，例如第1條提到從絕望到充滿希望的轉變，顯示其情緒容易受外界影響，特別是來自關心她的人。第17條提到「感到自己大部分時間都很可悲」，第88條則說「在求職過程中感到毫無價值」，這些都展現了她在面對生活困境時，容易陷入沮喪和自我否定。第51條和第44條也提到了「洩氣」和「沮喪」的情緒，與工作和生活壓力相關。

* **焦慮/不安 (Anxiety/Unease):** 第15條直接表達了「不安」的情緒，而第34條則描述了她對於人際關係和親密關係的焦慮，害怕無法被全然接受和安全，擔心隨之而來的性期待。第67條也提到她處於「情緒的雲霄飛車」，預感有不愉快的事情即將發生，展現了持續的焦慮感。

* **負面情緒的綜合表現：**  除了上述兩種情緒外，日記中還包含了其他負面情緒，例如：惱怒 (irritated, angered)、罪惡感 (greedy, wrong, selfish)、被拒絕感 (rejected)、壓力過大 (overwhelmed, burdened)、害怕 (terrified)、無聊 (listless)、失望 (dissatisfied)、自責 (fucked up, pathetic)、尷尬 (embarrassed)等等。這些負面情緒往往互相交織，例如第20條同時感受到惱怒和被拒絕，顯示她的情緒狀態複雜且多變。

* **正面情緒的偶爾出現：**  儘管負面情緒佔據了日記的主要部分，但作者偶爾也會體驗到正面情緒，例如：希望 (hopeful)、快樂 (happy, excited)、滿足感 (content, smug, achieved)、愛 (love)、感激 (grateful)、愉悅 (amused, delighted, pleasantly surprised)、浪漫 (romantic)、熱情 (passionate)、靈性體驗 (spiritual encounte

###練習2 (簡轉繁)

In [71]:
!pip install opencc-python-reimplemented pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 7.0 MB/s eta 0:00:00


In [101]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets3 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1b6WaMdMXtSRzg5PqQ9OXaw-32gAP7W0-yVhrxtsVL1A/edit?usp=sharing').get_worksheet(3)

In [102]:
# 讀取所有數據
rows = gsheets3.get_all_records()
import pandas as pd
df3 = pd.DataFrame(rows)

In [103]:
from opencc import OpenCC

# 建立簡體轉繁體的轉換器（s2t = 簡體轉繁體）
cc = OpenCC('s2t')

# 對文字欄位轉換簡體 → 繁體
for col in df3.columns:
    if df3[col].dtype == object:
        df3[col] = df3[col].apply(lambda x: cc.convert(x) if isinstance(x, str) else x)

# 顯示轉換後結果（可選）
df3.head()

,date,tag,headline,content
0,2016-01-01,詳細全文,陸軍領導機構火箭軍戰略支援部隊成立大會在京舉行 習近平向中國人民解放軍陸軍火箭軍戰略支援部隊...,中國人民解放軍陸軍領導機構、中國人民解放軍火箭軍、中國人民解放軍戰略支援部隊成立大會2015...
1,2016-01-01,詳細全文,中央軍委印發《關於深化國防和軍隊改革的意見》,經中央軍委主席習近平批准，中央軍委近日印發了《關於深化國防和軍隊改革的意見》。\n《意見》強...
2,2016-01-01,詳細全文,《習近平關於嚴明黨的紀律和規矩論述摘編》出版發行,由中共中央紀律檢查委員會、中共中央文獻研究室編輯的《習近平關於嚴明黨的紀律和規矩論述摘編》一...
3,2016-01-01,詳細全文,以實際行動向黨中央看齊 向高標準努力,廣大黨員幹部正在積極學習習近平總書記在中央政治局專題民主生活會上的重要講話。大家紛紛表示要把...
4,2016-01-01,詳細全文,【年終特稿】關鍵之年 改革挺進深水區,剛剛過去的2015年，是全面深化改革的關鍵之年。改革集中發力在制約經濟社會發展的深層次矛盾，...


In [104]:
df3.describe()

,date,tag,headline,content
count,20738,20738,20738,20738
unique,1008,3,20146,20610
top,2018-09-05,詳細全文,李克強主持召開國務院常務會議,
freq,42,11534,100,107


In [105]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [106]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

In [107]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [111]:
df3['Chinese_Content'] = df3['content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df3['Tokenized'] = df3['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words3 = [word for tokens in df3['Tokenized'] for word in tokens if len(word) > 1]

In [112]:
df3['Chinese_Content']

,Chinese_Content
0,中國人民解放軍陸軍領導機構中國人民解放軍火箭軍中國人民解放軍戰略支援部隊成立大會年月日在八一...
1,經中央軍委主席習近平批准中央軍委近日印發了關於深化國防和軍隊改革的意見意見強調黨的十八大以來...
2,由中共中央紀律檢查委員會中共中央文獻研究室編輯的習近平關於嚴明黨的紀律和規矩論述摘編一書近日...
3,廣大黨員幹部正在積極學習習近平總書記在中央政治局專題民主生活會上的重要講話大家紛紛表示要把踐...
4,剛剛過去的年是全面深化改革的關鍵之年改革集中發力在制約經濟社會發展的深層次矛盾集中發力在妨礙...
...,...
20733,土耳其官方媒體日前報道說根據俄土達成的在敘利亞伊德利卜建立非軍事區的協議敘利亞反政府武裝國民...
20734,國際貨幣基金組織北京時間今天發佈報告宣佈將今明兩年世界經濟增速預期下調至比此前月份的預測值低...
20735,日烏克蘭切爾尼戈夫地區一座軍火庫發生爆炸事發地位於烏克蘭首都基輔東部約公里處事發地周圍半徑公...
20736,聯合國政府間氣候變化專門委員會日發佈全球升溫特別報告報告顯示如果氣候變暖以目前的速度持續下去...


In [92]:
df3['Tokenized']

,Tokenized
0,"[中國, 人民, 解放, 軍陸, 軍, 領導, 機構, 中國, 人民, 解放, 軍, 火箭,..."
1,"[經, 中央, 軍委, 主席, 習近, 平, 批准, 中央, 軍委, 近日, 印發, 了關,..."
2,"[由, 中共中央, 紀律, 檢查, 委員會, 中共中央, 文獻, 研究室, 編輯, 的, 習..."
3,"[廣大黨員, 幹部, 正在, 積極學, 習習, 近平, 總書記, 在, 中央政治局, 專題,..."
4,"[剛剛, 過去, 的, 年, 是, 全面, 深化改革, 的, 關鍵, 之, 年, 改革, 集..."
...,...
20733,"[土耳其, 官方, 媒體, 日前, 報道, 說, 根據, 俄土, 達成, 的, 在, 敘利亞..."
20734,"[國際, 貨幣, 基金, 組織, 北京, 時間, 今天, 發, 佈, 報告, 宣, 佈, 將..."
20735,"[日, 烏克蘭, 切爾尼戈夫, 地區, 一座, 軍火庫, 發生, 爆炸, 事發, 地位, 於..."
20736,"[聯合國, 政府, 間, 氣候, 變化, 專門委員會, 日發, 佈, 全球, 升溫, 特別,..."


In [113]:
word_counts3 = Counter(all_words3)
word_counts3

Counter({'中國人民解放軍': 207,
         '陸軍領': 6,
         '導機構': 6,
         '火箭': 500,
         '戰略': 4158,
         '支援': 136,
         '部隊': 854,
         '成立': 1123,
         '大會': 1639,
         '年月日': 896,
         '八一': 100,
         '大樓': 78,
         '隆重': 133,
         '舉行': 3360,
         '中共中央': 1469,
         '總書': 1014,
         '記國家': 504,
         '主席': 7655,
         '中央': 9870,
         '軍委': 656,
         '習近': 6005,
         '平向陸': 1,
         '軍戰略': 20,
         '授予': 175,
         '軍旗': 24,
         '並致': 70,
         '訓詞': 19,
         '代表': 5134,
         '軍委向': 26,
         '同志': 2801,
         '全軍部隊': 10,
         '致以': 260,
         '熱烈': 570,
         '賀強': 2,
         '調要': 273,
         '堅持': 4443,
         '以黨': 69,
         '形勢': 1194,
         '強軍目標': 66,
         '引領': 953,
         '深入': 2548,
         '貫徹': 1519,
         '軍事': 963,
         '方針': 478,
         '全面': 6498,
         '實施': 2761,
         '改革': 7379,
         '強軍戰略': 18,
         '堅定': 1857,

In [114]:
stop_words3 = set(['所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個'])
filtered_word_counts3 = {word: count for word, count in word_counts3.items() if word not in stop_words3}

In [115]:
"""
對指定的詞語或片段進行替換，製作一份更新後的文字資料。
"""
jieba.add_word('中國人民解放軍')

In [116]:
# 將詞頻轉為 DataFrame 並排序
#word_freq_df3 = pd.DataFrame(filtered_word_counts3.items(), columns=['Word', 'Frequency'])
word_freq_df3 = pd.DataFrame(word_counts3.items(), columns=['Word', 'Frequency'])
word_freq_df3 = word_freq_df3.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df3.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前30名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [ ]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

In [117]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words3 = word_freq_df3.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words3['Word'], top_words3['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [121]:
# 按日期計算討論熱度
df3['Date'] = pd.to_datetime(df3['date'])
heatmap_data = df3.groupby('Date').size().reset_index(name='News_Count')

# 繪製趨勢圖
fig = px.line(heatmap_data, x='Date', y='News_Count',
              title='不同時間段的新聞量',
              labels={'Date': '日期', 'Discussion_Count': '新聞量'},
              markers=True)

fig.update_layout(
    xaxis_title="日期",
    yaxis_title="新聞量",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [118]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請摘要出這些新聞的重點，把最多提及的主題及其相關的內容整併在一起，說明這些新聞的風向：{df3.to_string()}"

response = model.generate_content(prompt)
print(response.text)

BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: unknown error

###**大家為什麼會來上這門課**
https://github.com/peculab/PythonAI4Beginners/issues/1

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1b6WaMdMXtSRzg5PqQ9OXaw-32gAP7W0-yVhrxtsVL1A/edit?usp=sharing').sheet1

In [ ]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1b6WaMdMXtSRzg5PqQ9OXaw-32gAP7W0-yVhrxtsVL1A/edit?usp=sharing').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [ ]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [ ]:
df.head()

,Account,Date,Cleaned_Content
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...
1,pupupeter,"Sep 2, 2024",Collaborator pupupeter • 高中生們兒早哇，我是這次的助教，我叫家瑋...
2,Junxian0829,"Sep 2, 2024",Collaborator Junxian0829 • 各位好，我是鄭鈞賢，是台師大機電系大...
3,tedlu-tw,"Sep 2, 2024",Collaborator tedlu-tw 各位好，和大家一樣是高中生，就讀北市數位實中，...
4,Eric970308-123,"Sep 2, 2024",Eric970308-123 大家好，我是胡程勛，目前高二，就讀彰化縣溪湖高中207班，在...


In [ ]:
df.describe()

,Account,Date,Cleaned_Content
count,312,312,312
unique,272,13,312
top,peculab,"Sep 5, 2024",Alan960609 commented 我是花蓮高工 杜謙祺 會想上這門課是因為想學習程式...
freq,11,110,1


In [ ]:
#Install packages in Google Colab
!pip install jieba
!pip install snownlp

In [62]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [63]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

In [64]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [65]:
df['Chinese_Content'] = df['Cleaned_Content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [66]:
df['Chinese_Content']

,Chinese_Content
0,學校姓名以及你為什麼來上這門課並留下你的第一個程式連結的我想要透過寫程式來實現我心中的想法我...
1,高中生們兒早哇我是這次的助教我叫家瑋現為台灣師範大學歷史系三年級的同學也是個愛大便的小孩子這...
2,各位好我是鄭鈞賢是台師大機電系大三的從前我的夢想是成為一名頂尖的歌手可惜因為一些個人因素我唱...
3,各位好和大家一樣是高中生就讀北市數位實中同時在臺大資工擔任研究助理雖然如此但我總是宣稱自己是...
4,大家好我是胡程勛目前高二就讀彰化縣溪湖高中班在班上擔任的幹部是設備股長下面的程式是我在高一的...
...,...
307,我是嘉工陳信錩希望可以學到東西
308,我是南港高中高廷翰對程式基礎有幫助
309,我是鳳梨大學傳播系的漁人在網路上路過一起來學的以下程式碼可以生成圖片喔是我在課上來的哈
310,我是鳳梨大學傳播系的漁人在網路上路過一起來學的以下程式碼可以生成圖片喔是我在課上來的哈


In [67]:
df['Tokenized']

,Tokenized
0,"[學校, 姓名, 以及, 你, 為什麼, 來上, 這門課, 並, 留下, 你, 的, 第一,..."
1,"[高中生, 們, 兒, 早, 哇, 我, 是, 這次, 的, 助教, 我, 叫, 家瑋, 現..."
2,"[各位, 好, 我, 是, 鄭鈞賢, 是, 台師大機電系, 大三, 的, 從, 前, 我, ..."
3,"[各位, 好, 和, 大家, 一樣, 是, 高中生, 就, 讀北市, 數位, 實中, 同時,..."
4,"[大家, 好, 我, 是, 胡程勛, 目前, 高二, 就, 讀, 彰化縣, 溪湖高中, 班,..."
...,...
307,"[我, 是, 嘉工, 陳信錩, 希望, 可以, 學到, 東西]"
308,"[我, 是, 南港高中, 高廷翰, 對, 程式, 基礎, 有, 幫助]"
309,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ..."
310,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ..."


In [68]:
word_counts = Counter(all_words)
word_counts

Counter({'學校': 5,
         '姓名': 2,
         '以及': 3,
         '為什麼': 2,
         '來上': 4,
         '這門課': 47,
         '留下': 1,
         '第一': 9,
         '程式': 143,
         '連結': 16,
         '想要': 29,
         '透過': 12,
         '實現': 2,
         '心中': 1,
         '想法': 1,
         '高中生': 3,
         '這次': 2,
         '助教': 7,
         '家瑋': 3,
         '現為': 1,
         '台灣師範大學': 1,
         '歷史系': 1,
         '三年': 1,
         '同學': 3,
         '大便': 1,
         '小孩子': 1,
         '這是': 13,
         '大家': 145,
         '整理': 2,
         '蔡老師': 2,
         '可以': 54,
         '觀覽': 1,
         '一下': 6,
         '下面': 3,
         '那位': 1,
         '冷面': 1,
         '笑匠': 1,
         '小心': 1,
         '嘿然後有': 1,
         '時候': 3,
         '我會': 5,
         '不正': 2,
         '搜哩嘿': 1,
         '拍謝': 1,
         '黑若': 1,
         '不想': 1,
         '直接': 1,
         '電腦': 5,
         '一關': 1,
         '機此': 1,
         '失效': 1,
         '上面': 2,
         '各位': 6,
         '鄭鈞賢': 1,
    

In [69]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [61]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [ ]:
word_freq_df

,Word,Frequency
0,高中,149
1,程式,143
2,希望,111
3,學習,62
4,可以,54
...,...,...
962,模型,1
963,弄出,1
964,哪天,1
965,工具,1


In [ ]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 47 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 0s (19.6 MB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 126213 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [ ]:
# 按日期計算討論熱度
df['Date'] = pd.to_datetime(df['Date'])
heatmap_data = df.groupby('Date').size().reset_index(name='Discussion_Count')

# 繪製趨勢圖
fig = px.line(heatmap_data, x='Date', y='Discussion_Count',
              title='不同時間段的討論熱度趨勢',
              labels={'Date': '日期', 'Discussion_Count': '討論數量'},
              markers=True)

fig.update_layout(
    xaxis_title="日期",
    yaxis_title="討論數量",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [ ]:
from google.colab import userdata
apikey = userdata.get('test')

In [ ]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請摘要出這些留言的重點，並補上重點中的情緒表現，把最多人呈現的情緒及其相關的內容整併在一起，說明這些發文的風向：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這些留言主要來自高中生，以及少數大學生、老師和社會人士，他們都參與了一個 Python 與 AI 的入門課程。留言展現了**高度的積極性和期待感**，也帶有一些**興奮和好奇**。

**主要情緒與內容：**

* **積極期待、興奮好奇 (佔大多數)：**  絕大多數留言者表達了對程式設計、AI 的濃厚興趣，並期待透過這個課程學習相關知識，提升技能，應用於未來學習、工作或生活。例如，許多學生提到想學習程式設計以幫助未來升學、就業，或將其應用於生活，甚至想利用AI來提升效率或解決問題。  他們使用大量的正面詞語，例如「有興趣」、「希望」、「期待」、「想學習」、「很厲害」、「很有趣」、「很棒」等。  部分留言中甚至展現了對 AI 的憧憬，例如想用 AI 幫自己寫作業（雖然多數是玩笑話），或用程式碼賺錢。

* **害怕與不安 (少量)：** 少數留言者表達了對課程難度的擔憂，例如「覺得很難」、「看起來有點難」，但也同時展現了他們願意嘗試和克服困難的積極態度，例如「我會加油」、「我會嘗試看看」。

* **分享學習成果 (少量)：** 部分學生分享了他們之前學習程式設計的經驗和作品，展現了他們的學習成果，也為其他同學提供了參考。

* **助教的互動 (少量)：** 助教的留言主要在提醒學生開啓共用權限，並鼓勵同學們跨校交流。其語氣輕鬆活潑，也營造了良好的學習氛圍。


**風向總結：**

整體發文風向非常正面積極。學生們對課程充滿期待，對 AI 和程式設計抱持著濃厚的學習慾望。雖然少數人表達了對難度的擔憂，但整體而言，學習的熱情和積極的態度佔據了主導地位。  這也反映出 AI 和程式設計在年輕一代中日益普及的趨勢，以及他們對未來科技發展的關注和期待。

